# Process ODH Data
This notebook extracts the necessary numbers for PDE DSA from the output of my parseODH.py script.

In [ ]:
using CSV,DataFrames,Dates,Plots,Measures,ColorSchemes,Printf
gr();clr=palette(:default,rev=true); default(palette=clr);

include("bspl1d.jl");

ENV["COLUMNS"]=10000;

## Inputs

In [ ]:
# Inputs
day0 = Date("2020-11-15");
dayf = Date("2021-01-15");

# Ohio age population
agekeys = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]
ohiopop = Dict{String,Float64}(
           "0-9"=>695933. + 1/3*2233431,
          "10-19"=>2. /3*2233431,
          "20-29"=>1554324.,
          "30-39"=>1428941.,
          "40-49"=>691199. + 1/2*1544429,
          "50-59"=>1/2*1544429. + 834920,
          "60-69"=>765841. + 631247,
          "70-79"=>449394. + 331259,
          "80+"=>228889. + 252072);
totpop = sum([ohiopop[key] for key in keys(ohiopop)]);
println("Total Ohio Population: $totpop")

# Normalizations from case counts to probability densities for comparing to pde prediction
#  At t=0 assumed only yˢ starts with nonzero values at time t=0
tsim = getfield(dayf-day0,:value);
ηyˢ = 1/totpop/(100*365+tsim);
ηyᵛ = 1/totpop/tsim;
ηyⁱ = 1/totpop/tsim;

## Plot formatting

In [ ]:
# Plot formatting
#  Finding that to get right output dimensions and font sizes for gr have to manually
#  compute scaling factors based on an observed print output
λplot = 100*7.5/10.42; λfont = 12/18; #scaling factors

# Final output two panel figure dimensions
figwdth = 7.45; fighght = 3.25; # inches

# Final output figure fontsizes
figtickfontsize = 12; figguidefontsize = 14; figtitlefontsize = 16; figlegendfontsize=8;

In [ ]:
lnopt = [:solid, :dash, :dot, :dashdot, :dashdotdot];
mkopt = [:none, :auto, :circle, :rect, :star5, :diamond, :hexagon, :cross, 
         :xcross, :utriangle, :dtriangle, :rtriangle, :ltriangle, :pentagon, 
         :heptagon, :octagon, :star4, :star6, :star7, :star8, :vline, :hline, 
         :+, :x]; 

## Function routines for processing

In [ ]:
function mymvavg(v::Vector{Float64})
    w = similar(v);
    for i=1:length(v)
        a = i-3<1 ? 1 : i-3;
        b = i+3>length(v) ? length(v) : i+3;
        
        w[i] = sum(v[a:b])/length(v[a:b]);
    end
    return w
end;

## Data
#### ODH cases

In [ ]:
dfodh = CSV.read("ODH_0119.csv",DataFrame);
println("Raw ODH:");
first(dfodh,3)

In [ ]:
flag = (dfodh[!,:time].>=day0).&(dfodh[!,:time].<=dayf);
dftemp = dfodh[flag,:]; 
dfwave = deepcopy(dftemp);
for i=2:12
    dfwave[!,i]=mymvavg(dftemp[!,i]);
end
println("7-day averaged ODH:")
first(dfwave,3)

In [ ]:
p1 = plot(dfodh[!,:time],dfodh[!,:daily_confirm],linewidth=1,linestyle=:dash,labels="",xlabel="date",ylabel="reported",
     title="ODH daily cases");
plot!(dftemp[!,:time],dftemp[!,:daily_confirm],linewidth=2,labels="");

p2 = plot(dftemp[!,:time],dftemp[!,:daily_confirm],linewidth=1,linestyle=:dash,labels="reported",xlabel="date",ylabel="")
plot!(dftemp[!,:time],dfwave[!,:daily_confirm],linewidth=2,labels="7-day avg",title="First wave");

lay = @layout [a b]
plot(p1,p2,layout=lay,size=(figwdth*λplot,fighght*λplot),margin=1mm,
     tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
     titlefontsize=round(figtitlefontsize*λfont),legendfontsize=round(figlegendfontsize*λfont))

In [ ]:
p3 = plot(dfwave[!,"time"],[dfwave[:,i] for i=2:10],legend=:topleft,size=(950,300),linewidth=2,
          labels=["0-9" "10-19" "20-29" "30-39" "40-49" "50-59" "60-69" "70-79" "80+"],
          title="ODH 7-day average",xlabel="date",ylabel="daily cases",margin=5mm,
          palette=clr)
plot!(xticks=day0:Day(14):dayf)

In [ ]:
CSV.write("ODH_snipdaily.csv",dfwave);

In [ ]:
dfcumwave = similar(dfwave); dfcumwave[!,:time]=dfwave[:,:time];
for key in agekeys
    dfcumwave[!,key]=cumsum(dfwave[!,key]);
end 
p4 = plot(dfcumwave[!,"time"],[dfcumwave[:,i] for i=2:10],legend=:topleft,size=(950,300),linewidth=2,
          labels=["0-9" "10-19" "20-29" "30-39" "40-49" "50-59" "60-69" "70-79" "80+"],
          title="ODH 7-day average",xlabel="date",ylabel="cumulative cases",margin=5mm,
          palette=clr)
plot!(xticks=day0:Day(14):dayf)

#### CDC vaccination

In [ ]:
dfcdc = CSV.read("VaxCdcJan.csv",DataFrame); 
rename!(dfcdc,"Date"=>"time","19-Oct"=>"10-19"); dfcdc[!,"0-9"]=convert(Vector{Float64},dfcdc[!,"0-9"]);
dfcdc[!,:time]=Date.(dfcdc[!,:time],"m/d/yyyy");
dfcdc[!,:time].=dfcdc[!,:time].-Day(14); # my parse vax routine added a 14 day delay
flag = (dfcdc[!,:time].>=day0).&(dfcdc[!,:time].<=dayf);
dfvax = dfcdc[flag,:];
println("'Raw' CDC Vax_1D data:")
last(dfvax,3)

In [ ]:
for i=2:10
    dfvax[!,i] = mymvavg(dfvax[!,i]);
end
println("7-day smooth CDC Vax_1D data:")
first(dfvax,3)

In [ ]:
p5 = plot(dfvax[!,"time"],[dfvax[:,i] for i=2:10],legend=:topleft,size=(950,300),linewidth=2,
          labels=["0-9" "10-19" "20-29" "30-39" "40-49" "50-59" "60-69" "70-79" "80+"],
          title="CDC 1D 7-day average",xlabel="date",ylabel="daily doses",margin=5mm,
          palette=clr);
println("Does this suggest a λ shape?")
vline!([day0],linewidth=0,labels="")
plot!(xticks=day0:Day(14):dayf)

In [ ]:
dfcumvax = similar(dfvax); dfcumvax[!,:time]=dfvax[:,:time];
for key in agekeys
    dfcumvax[!,key]=cumsum(dfvax[!,key]);
end 
p6 = plot(dfcumvax[!,"time"],[dfcumvax[:,i] for i=2:10],legend=:topleft,size=(950,300),linewidth=2,
          labels=["0-9" "10-19" "20-29" "30-39" "40-49" "50-59" "60-69" "70-79" "80+"],
          title="ODH 7-day average",xlabel="date",ylabel="cumulative vaccinations",margin=5mm,
          palette=clr,
          linestyle=reshape([lnopt[ℓ%5 + 1] for ℓ=2:10],1,9),
          marker=reshape([(i-2)÷5==0 ? :none : :utriangle for i=2:10],1,9),
          markeralpha=0.2,markersize=3.5)
vline!([day0],linewidth=0,labels="")
plot!(xticks=day0:Day(14):dayf,size=(figwdth*λplot,fighght*λplot),margin=1mm,
     tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
     titlefontsize=round(figtitlefontsize*λfont),legendfontsize=round(figlegendfontsize*λfont))
function yfmtr(x::Real)
    if x==0
        return "0.0e00"
    end
    snew = @sprintf "%1.1e" x
    return snew
end
plot!(yformatter=yfmtr)

### Vaccination hazard

We may estimate the hazard through the relation $\lambda(t)=\lim_{\Delta t\rightarrow 0}\frac{P\left(T\leq t+\Delta t\left.\right| T>t\right)}{\Delta t} = -\frac{d}{dt}\left[\ln G(t)\right]$ where $G(t) = P(T\geq t)$ is the survival function. We then estimate $\ln G(t)$, fit with a smooth spline, and compute the appropriate derivative.

In [ ]:
# Parameters for spline knots
Δ = convert(Vector{Float64},LinRange(0.0,length(dfcumvax[!,"time"])-1,4));
Δ = sort!([Δ;5.0])
r = 2; d = 3;

In [ ]:
# Compute the ln survival function in each age group and extract empirical hazard
ℓnG = Dict{String,Vector{Float64}}();
for key in agekeys
    ℓnG[key] = (ohiopop[key] .- dfcumvax[!,key])/ohiopop[key] |> (x->log.(x));
end

bspl = Dict{String,Bspl1d}();
xval = convert(Vector{Float64},0:(length(dfcumvax[!,"time"])-1));

knots = Knots(Δ,r,d);
for key in agekeys
    bspl[key] = lsqspl(xval,-ℓnG[key],knots);
end

vbspl = [bspl[key] for key in agekeys];
# Note on p1 can do p1.<tab> etc to explore dropdown
p1 = plot(vbspl;npts=26); 
p1[1][1][:label]="0-9"; p1[1][1][:linestyle]=lnopt[3];
p1[1][2][:label]="10-19"; p1[1][2][:linestyle]=lnopt[4];
p1[1][3][:label]="20-29"; p1[1][3][:linestyle]=lnopt[5];
p1[1][4][:label]="30-39"; p1[1][4][:linestyle]=lnopt[1];
p1[1][5][:label]="40-49"; p1[1][5][:linestyle]=lnopt[2];
p1[1][6][:label]="50-59"; p1[1][6][:linestyle]=lnopt[3]; p1[1][6][:markershape]=:utriangle; p1[1][6][:markeralpha]=0.2; p1[1][6][:markersize]=3.5;
p1[1][7][:label]="60-69"; p1[1][7][:linestyle]=lnopt[4]; p1[1][7][:markershape]=:utriangle; p1[1][7][:markeralpha]=0.2; p1[1][7][:markersize]=3.5;
p1[1][8][:label]="70-79"; p1[1][8][:linestyle]=lnopt[5]; p1[1][8][:markershape]=:utriangle; p1[1][8][:markeralpha]=0.2; p1[1][8][:markersize]=3.5;
p1[1][9][:label]="80+"; p1[1][9][:linestyle]=lnopt[1]; p1[1][9][:markershape]=:utriangle; p1[1][9][:markeralpha]=0.2; p1[1][9][:markersize]=3.5;
for i=1:length(agekeys)
    key = agekeys[i];
    plot!(xval,-ℓnG[key],seriestype=:scatter,markersize=1.5,labels="",alpha=0.75,color=p1[1][i][:markercolor])
end
plot!(legend=:topleft,xlabel="days after Dec 15",ylabel="-ℓn P(T>=t)")

∂vbspl = [∂(bspl[key]) for key in agekeys];
p2 = plot(∂vbspl;npts=26);
p2[1][1][:label]="0-9"; p2[1][1][:linestyle]=lnopt[3];
p2[1][2][:label]="10-19"; p2[1][2][:linestyle]=lnopt[4];
p2[1][3][:label]="20-29"; p2[1][3][:linestyle]=lnopt[5];
p2[1][4][:label]="30-39"; p2[1][4][:linestyle]=lnopt[1];
p2[1][5][:label]="40-49"; p2[1][5][:linestyle]=lnopt[2];
p2[1][6][:label]="50-59"; p2[1][6][:linestyle]=lnopt[3]; p2[1][6][:markershape]=:utriangle; p2[1][6][:markeralpha]=0.2; p2[1][6][:markersize]=3.5;
p2[1][7][:label]="60-69"; p2[1][7][:linestyle]=lnopt[4]; p2[1][7][:markershape]=:utriangle; p2[1][7][:markeralpha]=0.2; p2[1][7][:markersize]=3.5;
p2[1][8][:label]="70-79"; p2[1][8][:linestyle]=lnopt[5]; p2[1][8][:markershape]=:utriangle; p2[1][8][:markeralpha]=0.2; p2[1][8][:markersize]=3.5;
p2[1][9][:label]="80+"; p2[1][9][:linestyle]=lnopt[1]; p2[1][9][:markershape]=:utriangle; p2[1][9][:markeralpha]=0.2; p2[1][9][:markersize]=3.5;
plot!(xlabel="days after Dec 15th",ylabel="hazard rate λ",legend=:topleft,ylim=(0.0,0.014));

lay = @layout [a b];
plot(p1,p2,layout=lay,size=(figwdth*λplot,fighght*λplot),margin=1mm,
     tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
     titlefontsize=round(figtitlefontsize*λfont),legendfontsize=round(figlegendfontsize*λfont))

In [ ]:
savefig("VaxHaz.pdf");

In [ ]:
lay = @layout [a;b c]
plot!(p6,title="");
plot(p6,p1,p2,layout=lay,size=(figwdth*λplot,2*fighght*λplot),margin=1mm,
     tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
     titlefontsize=round(figtitlefontsize*λfont),legendfontsize=round(figlegendfontsize*λfont))

In [ ]:
savefig("mst_vax.pdf");

In [ ]:
println("Hazard rate bsplines by age group:")
Pλ = Vector{Any}(undef,9);
for i=1:9
    #println(key);
    #println(∂(bspl[key]))
    #println("")
    
    plot(∂(bspl[agekeys[i]]));
    Pλ[i] = plot!(xlabel="days after Dec 15th",ylabel="λ",legend=:topleft,ylim=(0.0,0.014));
    Pλ[i][1][1][:label]=labels=agekeys[i];
end

lay = @layout [a b c;d e f;g h i]
plot(Pλ[1],Pλ[2],Pλ[3],Pλ[4],Pλ[5],Pλ[6],Pλ[7],Pλ[8],Pλ[9],layout=lay,size=(900,600))

In [ ]:
savefig("VaxHaz_ages.pdf");

### Susceptible Population Density
#### $f^s$

In [ ]:
function fˢ(s::Float64)
    if (s<0.0)||(s>100.0)
        return 0.0
    elseif s<70.0
        return 0.0125/1.0625
    else
        return (s-100)*(-0.0125/30)/1.0625
    end
end;

In [ ]:
agsz = [ohiopop[key]/totpop for key in ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"]];
p7 = plot([convert(Vector,5:10:75);[90]],[agsz[1:end-1]/10;agsz[end]/20] ,seriestype=:bar,ylabel="pdf",xlabel="age group",
          xticks=0:10:100,size=(475,200),margin=5mm,labels="",title="Ohio age distribution",
          bins=[0:10:80,100],palette=palette(:default),alpha=0.5,
          linecolor=:white,linewidth=0);
s=0:0.01:100; y=fˢ.(s);
println("Except numerically we also multiply by switch bumps at s=0,s=100:")
plot!(s,y,linewidth=3,c=palette(:default)[1],labels="fˢ",size=(figwdth*λplot*.75,fighght*λplot),margin=1mm,
     tickfontsize=round(figtickfontsize*λfont),guidefontsize=round(figguidefontsize*λfont),
     titlefontsize=round(figtitlefontsize*λfont),legendfontsize=round(10*λfont))
#vline!([70.0],c=:red,linestyle=:dash,linewidth=1,labels="")
plot!(title="")
function yfmtr(x::Real)
    if x==0
        return "0.0e00"
    end
    snew = @sprintf "%1.1e" x
    return snew
end
plot!(yformatter=yfmtr)

In [ ]:
savefig("fs.pdf");